In [ ]:
"""
Colab Notebook Set Up

Use this cell to upload your kaggle.json file as well as the `download_data.sh`
and `preprocess.py` scripts.
"""

from google.colab import files
import os

# Upload files
kaggle = files.upload()
data_script = files.upload()
preprocess_script = files.upload()
config = files.upload()

# Verify uploads
for file in ["kaggle.json", "download_data.sh", "preprocess.py", "oct.yaml"]:
    assert file in os.listdir(), f"Make sure you upload the {file} file"

# Shell commands
!mkdir -p ~/.kaggle/ data/ models/ config/ scripts/
!mv kaggle.json ~/.kaggle/
!mv download_data.sh preprocess.py scripts/
!mv oct.yaml config/
!chmod 600 ~/.kaggle/kaggle.json
!chmod +x scripts/download_data.sh scripts/preprocess.py
!pip install -q kaggle pretrainedmodels rich

# Run shell commands
!scripts/download_data.sh
!python scripts/preprocess.py --config config/oct.yaml

In [152]:
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
import pretrainedmodels as models

from datetime import datetime
from tqdm.auto import tqdm
from rich import print
from glob import glob

import sys; sys.path.append(".")
from net import train

import numpy as np
import torch
import yaml
import os
import re

config = 'config/oct.yaml'

with open(config, "r") as file:
    config = yaml.safe_load(file)

In [138]:
class FineTuned(nn.Module):
    """
    Fine-tuned Output Layer for InceptionV3
    """

    def __init__(self, config):
        super(FineTuned, self).__init__()

        # self.ptm = models.__dict__[config["model-name"]](num_classes=1000, pretrained="imagenet")
        self.fc = nn.Sequential(
            nn.Linear(2048, config['num-classes']),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.fc(x)
        return x


# Create datasets
train_ds = train.FeatureDataset(config['features']['train'])
test_ds  = train.FeatureDataset(config['features']['test'])
val_ds   = train.FeatureDataset(config['features']['val'])

# Create dataloaders
train_dl = DataLoader(train_ds, batch_size=config['batch-size'], shuffle=True)
test_dl  = DataLoader(test_ds, batch_size=config['batch-size'], shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=config['batch-size'], shuffle=False)

# Instantiate model
model = FineTuned(config)
model.to(train.device())

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config['lr'])

/home/muaddib/.conda/envs/torch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/muaddib/.conda/envs/torch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/muaddib/.conda/envs/torch/lib/python3.10/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [153]:
def train_one_epoch(epoch_idx, writer):
    running_loss = 0.0
    last_loss = 0.0

    for i, data in enumerate(train_dl):
        inputs, labels = data
        inputs = inputs.to(train.device())
        labels = labels.to(train.device())

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Make predictions
        outputs = model(inputs)

        # Compute loss and its gradient
        loss = criterion(outputs, labels)
        loss.backward()

        # Adjust weights
        optimizer.step()

        # Gather and report statistics
        running_loss += loss.item()
        if i % 20 == 19:  # Print every 10 mini-batches
            last_loss = running_loss / 10  # loss per batch
            print(f'    batch {i+1} loss: {last_loss:.3f}')
            
            tb_x = epoch_idx * len(train_dl) + i + 1
            writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.0

    return last_loss

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(val_dl):
        vinputs, vlabels = vdata
        vinputs = vinputs.to(train.device())
        vlabels = vlabels.to(train.device())

        voutputs = model(vinputs)
        vloss = criterion(voutputs, vlabels)
        running_vloss += vloss.item()

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch for both training and validation
    writer.add_scalars(
        'Training vs. Validation Loss',
        { 'Training' : avg_loss, 'Validation' : avg_vloss },
        epoch_number + 1
    )
    writer.flush()

    # Track best performance:
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = f'models/oct_cnn_{timestamp}_{epoch_number}.pth'
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

In [ ]:
def train_one_epoch(epoch_idx, writer):
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for i, data in enumerate(train_dl):
        inputs, labels = data
        inputs = inputs.to(train.device())
        labels = labels.to(train.device())

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Make predictions
        outputs = model(inputs)

        # Compute loss and its gradient
        loss = criterion(outputs, labels)
        loss.backward()

        # Adjust weights
        optimizer.step()

        # Gather and report statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        if i % 20 == 19:  # Print every 10 mini-batches
            accuracy = correct_predictions / total_predictions
            last_loss = running_loss / 10  # loss per batch
            print(f'    batch {i+1} loss: {last_loss:.3f}, accuracy: {accuracy:.3f}')

            tb_x = epoch_idx * len(train_dl) + i + 1
            writer.add_scalar('Loss/train', last_loss, tb_x)
            writer.add_scalar('Accuracy/train', accuracy, tb_x)
            running_loss = 0.0

    return last_loss, accuracy


In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

In [ ]:
EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss, avg_acc = train_one_epoch(epoch_number, writer)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for i, vdata in enumerate(val_dl):
        vinputs, vlabels = vdata
        vinputs = vinputs.to(train.device())
        vlabels = vlabels.to(train.device())

        voutputs = model(vinputs)
        vloss = criterion(voutputs, vlabels)
        running_vloss += vloss.item()

        _, predicted = torch.max(voutputs.data, 1)
        total_predictions += vlabels.size(0)
        correct_predictions += (predicted == vlabels).sum().item()

    avg_vloss = running_vloss / (i + 1)
    avg_vacc = correct_predictions / total_predictions
    print('LOSS train {} valid {}. ACCURACY train {} valid {}'.format(avg_loss, avg_vloss, avg_acc, avg_vacc))

    # Log the running loss averaged per batch for both training and validation
    writer.add_scalars(
        'Training vs. Validation Loss',
        { 'Training' : avg_loss, 'Validation' : avg_vloss },
        epoch_number + 1
    )
    writer.add_scalars(
        'Training vs. Validation Accuracy',
        { 'Training' : avg_acc, 'Validation' : avg_vacc },
        epoch_number + 1
    )
    writer.flush()

    # Track best performance:
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = f'models/oct_cnn_{timestamp}_{epoch_number}.pth'
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

In [144]:
train_losses, val_losses = [], []

for epoch in range(config['epochs']):
    running_loss = 0.0
    running_acc = 0.0

    # Training
    model.train()
    pbar = train.pbar(train_dl)

    for i, (inputs, labels) in pbar:
        inputs = inputs.to(train.device())
        labels = labels.to(train.device())

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        acc = train.accuracy(outputs, labels)
        running_acc += acc

        pbar.set_description(f"Epoch [{epoch + 1}/{config['epochs']}]")
        pbar.set_postfix(loss=loss.item(), accuracy=acc)

    train_loss = running_loss / len(train_dl)
    train_losses.append(train_loss)
    train_acc = running_acc / len(train_dl)

    # Validation
    val_loss = 0.0
    val_acc = 0.0
    model.eval()

    with torch.no_grad():
        pbar = train.pbar(val_dl)

        for i, (inputs, labels) in pbar:
            inputs = inputs.to(train.device())
            labels = labels.to(train.device())

            outputs = model(inputs)
            batch_loss = criterion(outputs, labels)

            val_loss += batch_loss.item()
            acc = train.accuracy(outputs, labels)
            val_acc += acc

            pbar.set_description(f"Epoch [{epoch + 1}/{config['epochs']}]")
            pbar.set_postfix(loss=batch_loss.item(), accuracy=acc)

    val_loss /= len(val_dl)
    val_losses.append(val_loss)
    val_acc /= len(val_dl)

    print(f"Epoch {epoch+1}/{config['epochs']}.. "
          f"Train loss: {train_loss:.3f}.. "
          f"Train accuracy: {train_acc:.3f}.. "
          f"Validation loss: {val_loss:.3f}.. "
          f"Validation accuracy: {val_acc:.3f}")

20